<a href="https://colab.research.google.com/github/SauravMaheshkar/blob/main/examples/notebooks/samv2_automatic_segmentation_with_wandb_tables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Please add a `W&B` named secret containing your API Key to Colab.

## 📦 Packages and Basic Setup
---

In [ ]:
%%capture
!pip install git+https://github.com/SauravMaheshkar/samv2.git wandb
!wget https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_tiny.pt

url = "https://github.com/SauravMaheshkar/SauravMaheshkar/blob/main/assets/text2img/llama_spiderman_coffee.png?raw=true"

In [ ]:
import os

import wandb
from google.colab import userdata

os.environ["WANDB_API_KEY"] = userdata.get("W&B")

run = wandb.init(project="samv2", entity="sauravmaheshkar")

columns = ["image", "mask"]
wandb_table = wandb.Table(columns=columns)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import requests
from PIL import Image

image = Image.open(requests.get(url, stream=True).raw)
image = np.array(image.convert("RGB"))

In [ ]:
plt.imshow(image)

In [ ]:
from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator
from sam2.build_sam import build_sam2
from sam2.utils.misc import variant_to_config_mapping
from sam2.utils.visualization import show_masks

model = build_sam2(
    variant_to_config_mapping["tiny"],
    "/content/sam2_hiera_tiny.pt",
)

mask_generator = SAM2AutomaticMaskGenerator(model)

In [ ]:
masks = mask_generator.generate(image)

In [ ]:
output_mask = show_masks(
    image=image, masks=masks, scores=None, only_best=False, autogenerated_mask=True
)

In [ ]:
output_mask

In [ ]:
wandb_table.add_data(wandb.Image(image), wandb.Image(output_mask))

In [ ]:
run.log({"samv2_automatic_mask_generation": wandb_table})

wandb.finish()